In [ ]:
import os
from dotenv import load_dotenv
from lib.rag.util import rule_to_splits, splits_to_chromadb, read_chromavector, read_pgvector
from agent.rule_rag import init_chromavectorstore, init_pgtorstore

load_dotenv()
RULE_DIR = os.getenv('RULE_DIR')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
PERSIST_DIRECTORY = os.getenv('PERSIST_DIRECTORY')
CONNECTION_STRING = os.getenv('RAG_DB_URL')


In [ ]:
# croma vectore 초기화
init_chromavectorstore()

--- 1. initialize vector store ---
--- rule dir : ./rules
--- split file : ./rules/기본문서_rule.txt / 기본문서
****** loading file : ./rules/기본문서_rule.txt
****** spliting text
****** add metadata : 기본문서
--- split file : ./rules/프로젝트관리양식_rule.txt / 프로젝트관리양식
****** loading file : ./rules/프로젝트관리양식_rule.txt
****** spliting text
****** add metadata : 프로젝트관리양식
--- creating croma vector store
--- collection name : DOCUMENT_RULES
--- persist directory : ./vectorstore
****** creating croma vector store
****** complete croma vector store
--- 1. completed ---


In [5]:
# pgvector 초기화
init_pgtorstore()

--- 1. initialize vector store ---
--- rule dir : ./rules
--- split file : ./rules/기본문서_rule.txt / 기본문서
****** loading file : ./rules/기본문서_rule.txt
****** spliting text
****** add metadata : 기본문서
--- split file : ./rules/프로젝트관리양식_rule.txt / 프로젝트관리양식
****** loading file : ./rules/프로젝트관리양식_rule.txt
****** spliting text
****** add metadata : 프로젝트관리양식
--- creating croma vector store
--- collection name : DOCUMENT_RULES
--- persist directory : ./vectorstore
****** creating pgvector store
****** complete pgvector store
--- 1. completed ---


In [7]:
vectorstore = read_pgvector(collection_name=COLLECTION_NAME,connection_string=CONNECTION_STRING)
vectorstore

/mnt/c/myproject/wantedocs/lib/rag/util.py:54: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation.
  return PGVector(
/mnt/c/myproject/wantedocs/lib/rag/util.py:54: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update you

In [ ]:
store = vectorstore.CollectionStore()
store

<langchain_community.vectorstores.pgvector._get_embedding_collection_store.<locals>.CollectionStore at 0x739fdf256e90>

In [34]:
vectorstore.similarity_search("전체 문서",k=10)

[Document(metadata={'source': './rules/프로젝트관리양식_rule.txt', 'name': '프로젝트관리양식'}, page_content='# 프로젝트 관리 양식 검토 규칙\n\n## 1. 핵심 규칙 (문서 전체)\n\n* **[규칙 1-1: 플레이스홀더 금지]**\n    * [cite_start]문서 본문 전체에서 "여기에 텍스트를 입력하세요"라는 기본 문자열이 발견되어서는 안 됩니다. [cite: 8, 11, 14, 19, 22, 24, 27, 29, 33, 35, 37]\n    * 이 문자열이 발견될 경우, 해당 항목은 \'작성되지 않음\'으로 간주합니다.\n\n## 2. 페이지 1: 프로젝트 개요'),
 Document(metadata={'source': './rules/기본문서_rule.txt', 'name': '기본문서'}, page_content="# 문서 작성 가이드라인\n\n## 1. 제목 규칙\n    * 제목은 'HY 헤드라인 M' 글꼴, 20pt, 굵게(Bold) 처리해야 합니다.\n    * 제목에는 '보고서', '제안서' 등 문서 유형이 명확히 포함되어야 합니다.\n\n## 2. 작성자 정보\n    * '작성자:'와 '소속:' 항목이 본문 시작 전에 반드시 포함되어야 합니다.\n    * 작성 일자는 'YYYY-MM-DD' 형식으로 기재해야 합니다.\n\n## 3. 본문 스타일\n    * 모든 문장은 '-다.' 또는 '-함.' 체로 통일해야 합니다. ('-요.' 체 혼용 금지)\n    * 목차는 2단계를 초과하지 않아야 합니다. (예: 1. 가. / 2. 나.)"),
 Document(metadata={'source': './rules/프로젝트관리양식_rule.txt', 'name': '프로젝트관리양식'}, page_content="## 4. 페이지 3: 회고 (프로젝트 종료 시점)\n\n* **[규칙 4-1: 필수 섹션 (회고)]**\n    * '7) 회고 사항'은 '종료 후 진행' 항목입니다.

In [10]:
for doc in vectorstore.similarity_search(
    "피드백",
    k=10,
    filter={
        'name':'프로젝트관리양식'
}):
    print(f'\n\n****** meta : {doc.metadata["name"]}')
    print(doc.page_content)



****** meta : 프로젝트관리양식
## 3. 페이지 2: 프로젝트 관리

* **[규칙 3-1: 필수 섹션 (관리)]**
    * [cite_start]'4) 팀원별 수행 역할' 섹션이 구체적인 내용으로 채워져 있어야 합니다. [cite: 18]
* **[규칙 3-2: Daily Log]**
    * [cite_start]'5) 추진 사항 기록(daily)' 섹션에 최소 1개 이상의 일일 기록이 날짜("YY. MM. xx" 형식)와 함께 작성되어야 합니다. [cite: 20, 21]
* **[규칙 3-3: 피드백 기록]**
    * '6) 강사님 피드백' 섹션에 날짜와 함께 피드백 내용이 기록되어야 합니다. (피드백이 없었다면 '해당 없음' 등 명시적 표시 권장) [cite_start][cite: 25, 26]

## 4. 페이지 3: 회고 (프로젝트 종료 시점)


****** meta : 프로젝트관리양식
# 프로젝트 관리 양식 검토 규칙

## 1. 핵심 규칙 (문서 전체)

* **[규칙 1-1: 플레이스홀더 금지]**
    * [cite_start]문서 본문 전체에서 "여기에 텍스트를 입력하세요"라는 기본 문자열이 발견되어서는 안 됩니다. [cite: 8, 11, 14, 19, 22, 24, 27, 29, 33, 35, 37]
    * 이 문자열이 발견될 경우, 해당 항목은 '작성되지 않음'으로 간주합니다.

## 2. 페이지 1: 프로젝트 개요


****** meta : 프로젝트관리양식
## 4. 페이지 3: 회고 (프로젝트 종료 시점)

* **[규칙 4-1: 필수 섹션 (회고)]**
    * '7) 회고 사항'은 '종료 후 진행' 항목입니다. (만약 문서가 '진행 중' 상태라면 이 섹션은 비어 있을 수 있습니다.) [cite_start][cite: 31]
    * 만약 검토 시점이 '프로젝트 종료 후'라면, 다음 항목이 모두 채워져 있어야 합니다.
        * [cite_start]'잘한점' [cite: 32]
   

In [11]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k":10,
        "filter": {"name": "프로젝트관리양식"}
    }
)
docs = retriever.invoke('모든 규칙')
docs

[Document(metadata={'source': './rules/프로젝트관리양식_rule.txt', 'name': '프로젝트관리양식'}, page_content='# 프로젝트 관리 양식 검토 규칙\n\n## 1. 핵심 규칙 (문서 전체)\n\n* **[규칙 1-1: 플레이스홀더 금지]**\n    * [cite_start]문서 본문 전체에서 "여기에 텍스트를 입력하세요"라는 기본 문자열이 발견되어서는 안 됩니다. [cite: 8, 11, 14, 19, 22, 24, 27, 29, 33, 35, 37]\n    * 이 문자열이 발견될 경우, 해당 항목은 \'작성되지 않음\'으로 간주합니다.\n\n## 2. 페이지 1: 프로젝트 개요'),
 Document(metadata={'source': './rules/프로젝트관리양식_rule.txt', 'name': '프로젝트관리양식'}, page_content='* **[규칙 2-1: 기본 정보]**\n    * [cite_start]\'프로젝트 이름\'이 반드시 명시되어야 합니다. [cite: 4]\n    * [cite_start]\'트랙 / 팀명\' (예: AI트랙 / OOO팀)이 명시되어야 합니다. [cite: 5]\n    * [cite_start]\'팀원\' 목록이 명시되어야 합니다. [cite: 6]\n* **[규칙 2-2: 프로젝트 기간]**\n    * [cite_start]프로젝트 기간 (시작일과 종료일)이 "YY년 MM월 N일" 또는 유사한 날짜 형식으로 기재되어야 합니다. [cite: 3]\n* **[규칙 2-3: 필수 섹션 (개요)]**\n    * [cite_start]\'1) 기획 배경\' 섹션이 구체적인 내용으로 채워져 있어야 합니다. [cite: 7]\n    * [cite_start]\'2) 프로젝트 목표\' 섹션이 구체적인 내용으로 채워져 있어야 합니다. [cite: 10]'),
 Document(metadata={'source': './rules/프로젝트관리양식_rule.txt',